In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
df = dd.read_csv('E:/Technical_assessment/Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data_20240808.csv')

In [9]:

#['Global_Time'] = dd.to_datetime(df['Global_Time'], unit='ms')
#  Convert 'Global_Time' to datetime for easier manipulation
# df
# Define a function to identify valid car-following pairs
def find_valid_pairs(df):
    # Filter out rows where 'Following' is not zero
    following_df = df[df['Following'] != 0]
    
    # Sort by 'Vehicle_ID', 'Following', and 'Global_Time'
    following_df = following_df.sort_values(by=['Vehicle_ID', 'Following', 'Global_Time'])

    # Group by 'Vehicle_ID' and 'Following'
    grouped = following_df.groupby(['Vehicle_ID', 'Following'])
    
    # Function to calculate continuous following duration
    def calculate_following_duration(group):
        group = group.sort_values('Global_Time')
        group['Time_Diff'] = group['Global_Time'].diff().fillna(pd.Timedelta(seconds=0))
        
        # Sum of continuous following durations
        total_following_duration = group['Time_Diff'].sum().total_seconds()
        
        return total_following_duration
    
    # Apply the function and filter for valid pairs
    following_durations = grouped.apply(calculate_following_duration, meta=('x', 'f8'))
    valid_pairs = following_durations[following_durations >= 20]
    
    return valid_pairs

# Apply the function to the dataset
valid_pairs = find_valid_pairs(df)

# Compute the result
num_valid_pairs = valid_pairs.compute().shape[0]

# Print the number of valid car-following pairs
print(f"Number of valid car-following pairs (following for at least 20 seconds): {num_valid_pairs}")

Number of valid car-following pairs (following for at least 20 seconds): 15992


In [12]:
# Calculate total number of unique vehicles
total_vehicles = df['Vehicle_ID'].nunique().compute()

print(f"Total number of vehicles: {total_vehicles}")

Total number of vehicles: 3233
